In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train=pd.read_csv("/kaggle/input/titanic/train.csv")
train.head()

# train_data.isna().sum()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test=pd.read_csv("/kaggle/input/titanic/test.csv")

main_test=test[['Pclass','Sex','SibSp','Parch']]
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
sub=pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [5]:
y=train.Survived
X=train[['Pclass', 'Sex', 'SibSp', 'Parch']]


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.metrics import r2_score,mean_absolute_error,accuracy_score


In [7]:



# X_new=pd.get_dummies(X,columns=['Sex','Embarked'])
# X_new['Age']=X_new['Age'].fillna(X_new.Age.mean())
# model=RandomForestClassifier(random_state=42)
# model.fit(X_new,y)
# imp=pd.Series(model.feature_importances_,index=X_new.columns).sort_values(ascending=False)

# X_new.info()
# X_new.isna().sum()
# X_new

In [8]:

numeric_columns=['Pclass','SibSp','Parch']
categorical_columns=['Sex']

numeric_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])
categorical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])
preprocessor=ColumnTransformer(transformers=[
    ('num',numeric_transformer,numeric_columns),
    ('cat',categorical_transformer,categorical_columns)
])
pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',RandomForestClassifier(random_state=42))
])
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)


In [9]:
# params={
#     'model__n_estimators':[100,500,1000],
#     'model__max_depth':[3,5,7],
#     'model__learning_rate':[0.01,0.05,0.1],
#     'model__reg_lambda':[0,0.1,1],
#     'model__reg_alpha':[0,0.1,1],
#     'model__subsample':[0.5,0.7,0.8],
#     'model__colsample_bytree':[0.5,0.7,0.8]
# }
# randomized_search=RandomizedSearchCV(
#     estimator=pipeline,
#     param_distributions=params,
#     cv=5,
#     scoring='accuracy',
#     verbose=2,
#     random_state=43,
#     n_jobs=-1
# )


In [10]:
pipeline.set_params(
    model__n_estimators=100,
    model__max_depth=3
)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'SibSp', 'Parch']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex'])])),
                ('model',
                 RandomForestClassifier(max_depth=3, random_state=42))])

In [11]:
# pipeline.set_params(
#     model__random_state=42,
#     model__n_estimators=100,
#     model__max_depth=7,
#     model__learning_rate=0.1,
#     model__colsample_bytree=0.5,
#     model__subsample=0.5,
#     model__reg_lambda=0,
#     model__reg_alpha=0.1,
#     model__n_jobs=-1
# )

In [12]:
# randomized_search.fit(X_train,y_train)
# best_model=randomized_search.best_estimator_
# predict=best_model.predict(X_test)
# print("best params",randomized_search.best_params_)
# print("mean abs error",mean_absolute_error(y_test,predict))
# print("r2 score",r2_score(y_test,predict))

In [13]:
pipeline.fit(X_train,y_train)
predict=pipeline.predict(X_test)
print("accuracy score",accuracy_score(y_test,predict))
main_predict=pipeline.predict(main_test)

accuracy score 0.8116591928251121


In [14]:
submission=pd.DataFrame({
    'PassengerId': test.PassengerId,
    'Survived':main_predict
})
submission.to_csv("submissions.csv",index=False)